In [65]:
from statsbombpy import sb

df_camp = sb.competitions()
df_camp.head(5)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [69]:
df_matches = sb.matches(competition_id=16, season_id=4)
df_matches.head(3)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,22912,2019-06-01,21:00:00.000,Europe - Champions League,2018/2019,Tottenham Hotspur,Liverpool,0,2,available,scheduled,2022-11-28T14:45:35.628134,2021-06-13T16:17:31.694,13,Final,\tEstádio Cívitas Metropolitano,Damir Skomina,Mauricio Roberto Pochettino Trossero,Jürgen Klopp,1.1.0,2,2


In [2]:
# Crie uma função simples que receba uma ID de partida e retorne os dados brutos dessa partida utilizando a API.
from statsbombpy import sb

df_events = sb.events(match_id=22912)
df_events.head(3)

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,50_50,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tottenham Hotspur,38,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 433, 'lineup': [{'player': {'id'...",Liverpool,24,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Liverpool,24,00:00:00.000,Half Start,NaN


In [21]:
df_events.shot_type.unique()

array([nan, 'Open Play', 'Penalty'], dtype=object)

In [21]:
# EXERCICIO 3
# Desenvolva uma função que:
#   Identifique os eventos principais da partida (gols, assistências, cartões, etc.).
#   Utilize um modelo LLM para transformar esses eventos em uma sumarização textual descritiva.
#      Tool de leitura de dataframe > https://python.langchain.com/docs/integrations/tools/pandas/
# Certifique-se de que a saída seja clara e amigável, como:
# "O time A venceu o time B por 3 a 1. Os destaques foram os gols de João e Lucas, além de uma assistência de Ana."

import pandas as pd

def df_events_pre_processing(df_events: pd.DataFrame) -> pd.DataFrame:
    '''
    Função que recebe um DataFrame de eventos de uma partida e retorna um DataFrame com os eventos principais da partida.[Gols, assistências, cartões, etc.]

    Parâmetros:
    df_events: DataFrame - DataFrame de eventos de uma partida (obtido através da função sb.events(match_id=ID_DA_PARTIDA))
    '''

    colunas_desejadas = set(['second','period','team','player','pass_goal_assist','bad_behaviour_card','shot_type','shot_outcome'])
    colunas_no_df = set(df_events.columns)
    filtro_colunas = list(colunas_desejadas & colunas_no_df)

    # Filtrando o DataFrame para incluir apenas as colunas que estão presentes, pois algumas colunas não aparecem para todas as partidas
    X = df_events[filtro_colunas]

    melt_values_desejado = set(['pass_goal_assist','bad_behaviour_card','shot_outcome'])
    melt_values = list(melt_values_desejado & colunas_no_df)

    X = X.melt(id_vars=['second','period','team','player','shot_type'], value_vars=melt_values)
    X = X[X.value.notnull()]

    descartar = ['Saved','Wayward','Post','Off T','Off S']
    X = X[~X['value'].isin(descartar)]

    #Concatenar as colunas "Variable" e "Value", usando ponto e virgula como separador

    X['key'] = X['variable'] + ';' + X['shot_type'].astype(str) + ';' + X['value'].astype(str) 



    #Criando uma coluna única para os eventos
    event_map = {
        ('bad_behaviour_card;nan;Yellow Card'): 'Yellow Card',
        ('bad_behaviour_card;nan;Red Card'): 'Red Card',
        ('shot_outcome;Open Play;Blocked'): 'Shot Blocked',
        ('shot_outcome;Open Play;Goal'): 'Goal Scored',
        ('shot_outcome;Penalty;Goal'): 'Penalty Goal Scored',
        ('pass_goal_assist;nan;True'): 'Assist Pass'

    }

    period_map = {
        1: 'first half',
        2: 'second half'
    }

    # Aplicando a transformação
    X['play'] = X['key'].apply(lambda k: event_map.get(k, 'EVENTO NAO MAPEADO'))

    X['match_period'] = X['period'].apply(lambda k: period_map.get(k, 'EVENTO NAO MAPEADO'))

    #Ordem cronológica
    X = X.sort_values(['period','second']).reset_index()

    X = X.drop(['index','second','variable','period', 'value','shot_type','key'], axis=1)

    #Ordenar colunas
    df_main_events = X[['match_period','team','player','play']]

    return df_main_events
    

,match_period,team,player,play
0,first half,Liverpool,Jordan Brian Henderson,Shot Blocked
1,first half,Liverpool,Mohamed Salah,Shot Blocked
2,first half,Liverpool,Mohamed Salah,Penalty Goal Scored
3,first half,Liverpool,Mohamed Salah,Shot Blocked
4,second half,Liverpool,Mohamed Salah,Shot Blocked
5,second half,Tottenham Hotspur,Christian Dannemann Eriksen,Shot Blocked
6,second half,Liverpool,Fábio Henrique Tavares,Shot Blocked
7,second half,Tottenham Hotspur,Lucas Rodrigues Moura da Silva,Shot Blocked
8,second half,Tottenham Hotspur,Bamidele Alli,Shot Blocked
9,second half,Liverpool,Virgil van Dijk,Shot Blocked


In [31]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

google_api_key = os.getenv('GOOGLE_API_KEY')

# Faz a consulta
def agent_half_analyses(df: pd.DataFrame, half: str) -> tuple:
    ''' 
    Cria um agente para analisar um tempo da partida.

    Args:
    df: DataFrame com os eventos da partida.
    half: String com o tempo da partida a ser analisado, esperado os valores "firt half" ou "second half".
    '''


    prompt = f"""
        As a football commentator, summarize in the main events of the match. Follow the instructions bellow:
            1. The data received represent only the events of {half}
            2. It's mandatory to comment on all the goals scored and pennaltys.
            3. Pay attention to the chronological order in the index column, and emphasize the teams and players who made each play. 
            4. End the comment by stating the final score of the half, for example: Team A 1 - 2 Team B.
    """

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key)

    df = df[df['match_period'] == half]
    agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code = True)

    return agent, prompt


def match_summarizer(first_half_analysis: str, second_half_analysis: str, tom_narrativo: str) -> str:
    '''
    Gera um resumo final da partida, condensando os resumos de cada tempo.

    Args:
    first_half_analysis: Resumo gerado por LLM para o primeiro tempo da partida.
    second_half_analysis: Resumo gerado por LLM para o segundo tempo da partida.
    '''
    prompt_final = f"""
    Como um narrador de futebol, analise os resumos de cada tempo da partida e condense em um resumo final. Siga as instruções abaixo:
        1. Você está contando a análise via chat, sobre uma partida antiga. Utilize um tom {tom_narrativo}
        2. Os resumos estão em inglês, porém você deve responder em português.
        3. Deixe claro quais eventos foram do primeiro e do segundo tempo.
        4. É obrigatório citar cada gol e se foram feitos por penalti.
        5. Cite os jogadores e times envolvidos em cada jogada.
        6. O placar final corresponde à soma do placar de cada tempo da partida.
        7. Finalize o resumo citando o placar final da partida.

    Resumo Primeiro tempo:
    {first_half_analysis}

    Resumo Segundo tempo:
    {second_half_analysis}
    """

    genai.configure(api_key=os.getenv('GEMINI_KEY'))
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt_final)

    return response.text

def invoke_lmms_match_analysis(df: pd.DataFrame, tom_narrativo: str) -> str:
    '''Invoca Agentes para analisar cada tempo da partida de forma separada e depois gera um resumo final

    Args:
    df: DataFrame com os eventos principais da partida, já filtrado e com tratamento dos labels dos eventos.
    '''

    agent,prompt = agent_half_analyses(df, 'first half')
    first_half_analysis = agent.invoke(prompt)
    first_half_analysis = first_half_analysis['output']

    agent,prompt = agent_half_analyses(df, 'second half')
    second_half_analysis = agent.invoke(prompt)
    second_half_analysis = second_half_analysis['output']

    final_response = match_summarizer(first_half_analysis, second_half_analysis, tom_narrativo)

    return final_response

invoke_lmms_match_analysis(df_main_events)



> Entering new AgentExecutor chain...
Thought: I need to analyze the provided dataframe to extract the relevant information for the football commentary.  The key events are goals and penalties, and I need to maintain chronological order.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
data = {'match_period': ['first half', 'first half', 'first half', 'first half'], 
        'team': ['Liverpool', 'Liverpool', 'Liverpool', 'Liverpool'], 
        'player': ['Jordan Brian Henderson', 'Mohamed Salah', 'Mohamed Salah', 'Mohamed Salah'], 
        'play': ['Shot Blocked', 'Shot Blocked', 'Penalty Goal Scored', 'Shot Blocked']}
df = pd.DataFrame(data)

commentary = ""
for index, row in df.iterrows():
    if "Goal" in row['play']:
        if "Penalty" in row['play']:
            commentary += f"Early on, {row['player']} of {row['team']} scored a penalty! "
        else:
            commentary += f"{row['player']} of {row['team']} found the back of the net! "
    elif "Sho

'Olá, pessoal! Estamos aqui para relembrar um jogaço que tivemos o prazer de narrar!  Vamos ao resumo dessa partida emocionante!\n\nNo primeiro tempo, vimos muita pressão do Liverpool.  Jordan Henderson e Mohamed Salah tiveram finalizações bloqueadas, mas o craque egípcio, Mohamed Salah, converteu um pênalti, abrindo o placar para os Reds!  O Liverpool foi para o intervalo com a vantagem mínima de 1 a 0.\n\nJá no segundo tempo, a partida ficou bem mais equilibrada.  Mais chances de gol, mas com muitas defesas! Vimos diversas finalizações bloqueadas:  Mohamed Salah (Liverpool), Christian Eriksen (Tottenham), Fábio Henrique Tavares (Liverpool), Lucas Moura (Tottenham) e Bamidele Alli (Tottenham) tiveram seus chutes defendidos.  Apesar da pressão do Tottenham, o placar não se alterou.\n\nPortanto, ao final de 90 minutos de muita emoção, Liverpool venceu por 1 a 0, com o gol marcado por Mohamed Salah, de pênalti, ainda no primeiro tempo.\n\nPlacar final: Liverpool 1 x 0 Tottenham Hotspur\n

In [44]:
########## CONTEUDO DO ARQUIVO AGENTS.PY ##########

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType
from dotenv import load_dotenv
import os
#from tools import *


load_dotenv()

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

if "GOOGLE_CSE_ID" not in os.environ:
    os.environ["GOOGLE_CSE_ID"] = os.getenv('GOOGLE_CSE_ID')

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    verbose=True,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)


def invoke_agent():
    '''
    This function creates an agent that can be used to interact with the user.

    Returns:
    agent_executor: AgentExecutor
    '''
    tools = instance_lang_chain_tools()

    ###### TROCAR PARA MEU NOVO PROMPT##########
    agent_prompt = hub.pull("musicindustrysearch/prompt")
    agent = create_react_agent(llm, tools, agent_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, max_turns=3)
    
    return agent_executor

    ##### NÃO PRECISA ALTERAR NADA AQUI#########
def ask_to_lmm(user_input):
    '''
    This function invoke the agent passing the question on user input.
    Args:
    user_input: str
    '''

    agent = invoke_agent()
    response = agent({"input": user_input})
    return response 

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'tools'

In [ ]:
# EXERCICIO 4
# Desenvolva uma funcionalidade que permita criar um perfil detalhado de um jogador específico da partida, contendo:
#   Nome do jogador.
#   Estatísticas como número de passes, finalizações, desarmes e minutos jogados.
#   Retorne os dados de forma clara e organizada (e.g., JSON ou tabela).

### ACHO QUE VOU FAZER COM SCRIPT E NÃO COM MODELO (VER  O QUE Q O FERNANDO FALOU NA AULA)

In [34]:
import pandas as pd

def player_stats(df: pd.DataFrame, nome_jogador: str):

    X = df[df['player'] == nome_jogador]

    stats = {
        'passes_completed': X[X['type'] == 'Pass'].shape[0],
        'passes_attempted': X[X['type'] == 'Pass'].shape[0],
        'shots': X[X['type'] == 'Shot'].shape[0],
        'shots_on_target':  X[(X['type'] == 'Shot') & (X['shot_outcome'] == 'On Target')].shape[0],
        'goals': X[(X['type'] == 'Shot') & (X['shot_outcome'] == 'Goal')].shape[0],
        'assistances': X[(X['type'] == 'Pass') & (X['pass_goal_assist'] == True)].shape[0],
        'fouls_committed': X[X['type'] == 'Foul'].shape[0],
        'fouls_won':  X[X['type'] == 'Foul Won'].shape[0],
        'tackles': X[X['type'] == 'Tackle'].shape[0],
        'interceptions': X[X['type'] == 'Interception'].shape[0],
        'dribbles_successful': X[(X['type'] == 'Dribble') & (X['dribble_outcome'] == 'Complete')].shape[0],
        'dribbles_attempted': X[X['type'] == 'Dribble'].shape[0]
    }

    return stats
	
player_stats(df_events, 'Mohamed Salah')

{'passes_completed': 17,
 'passes_attempted': 17,
 'shots': 6,
 'shots_on_target': 0,
 'goals': 1,
 'assistances': 0,
 'fouls_committed': 0,
 'fouls_won': 0,
 'tackles': 0,
 'interceptions': 0,
 'dribbles_successful': 0,
 'dribbles_attempted': 0}

In [ ]:
# EXERCICIO 5
# Desenvolvimento de uma API com FastAPI:
#       /match_summary: Retorna a sumarização de uma partida.
#       /player_profile: Retorna o perfil detalhado de um jogador.
# Utilize Pydantic para validar as entradas e saídas.

In [ ]:
# EXERCICIO 6
# Desenvolva uma funcionalidade para criar narrativas baseadas nos eventos da partida.
# Permita ao usuário escolher entre estilos de narração, como:
#       Formal (técnico e objetivo), Humorístico (descontraído e criativo), Técnico (análise detalhada dos eventos)

In [ ]:
#EXERCICIO 7
# Desenvolva uma interface simples que permita ao usuário:
#      Selecionar uma partida por ID.
#      Exibir os eventos principais e o perfil de um jogador.


###### CONTEUDO DO ARQUIVO PRINCIPAL
import streamlit as st
from utils import *

st.set_page_config(layout="wide")

st.sidebar.header('Navegação')

st.sidebar.header('Filtros')
filtros_barra_lateral()



In [ ]:
###### CONTEUDO A COLOCAR NO ARQUIVO UTILS.PY PARA CRIAR A BARRA DE FILTROS

def filtros_barra_lateral():

    #Inicializo as variáveis de sessão
    if 'filtro_camp' not in st.session_state:
        st.session_state.filtro_camp = 'Selecione'

    if 'filtro_ano' not in st.session_state:
        st.session_state.filtro_ano = 'Selecione'

    if 'filtro_partida' not in st.session_state:
        st.session_state.filtro_partida = 'Selecione'

    if 'filtro_id_partida' not in st.session_state:
        st.session_state.filtro_id_partida = 'Selecione'

    if 'df_partidas_filtrado' not in st.session_state:
        st.session_state.df_partidas_filtrado = pd.DataFrame()

    if 'df_eventos' not in st.session_state:
        st.session_state.df_eventos = pd.DataFrame()

    df_camp = sb.competitions()

    #Crio a lista de campeonatos
    lista_camp = df_camp['competition_name'].drop_duplicates().tolist()
    lista_camp.insert(0, 'Selecione')

    #Crio o Seletor
    st.session_state.filtro_camp = st.sidebar.selectbox('Filtrar Campeonato', lista_camp, key='1')

    #Aplico a condição para só filtrar se houver seleção
    if st.session_state.filtro_camp != 'Selecione':
        df_camp = df_camp.loc[df_camp['competition_name'] == st.session_state.filtro_camp]

        #lista dos anos
        lista_ano = df_camp['season_name'].drop_duplicates().tolist()
        lista_ano.insert(0, 'Selecione')
        #seletor de ano
        st.session_state.filtro_ano = st.sidebar.selectbox('Filtrar Ano', lista_ano)

        if st.session_state.filtro_ano != 'Selecione':
            df_camp = df_camp.loc[df_camp['season_name'] == st.session_state.filtro_ano]
            
            #Obtem o Dataset Partidas passando como parâmetro o id do campeonato e o id da temporada    
            df_partidas = sb.matches(competition_id=df_camp['competition_id'].values[0], season_id=df_camp['season_id'].values[0])

            #lista das partidas
            df_partidas['partida'] = df_partidas['home_team'] + ' x ' + df_partidas['away_team']
            df_partidas['resultado'] = df_partidas['home_score'].astype(str) + ' x ' + df_partidas['away_score'].astype(str)
            lista_partidas = df_partidas['partida'].drop_duplicates().tolist()
            lista_partidas.insert(0, 'Selecione')
            #seletor de partidas
            st.session_state.filtro_partida = st.sidebar.selectbox('Filtrar Partida', lista_partidas)

            if st.session_state.filtro_partida != 'Selecione':
                st.session_state.df_partidas_filtrado = df_partidas.loc[df_partidas['partida'] == st.session_state.filtro_partida]

                #Obtem o Dataset Eventos passando como parâmetro o id da partida
                filtro_id_partida = st.session_state.df_partidas_filtrado['match_id'].values[0]
                st.session_state.df_eventos = sb.events(match_id=filtro_id_partida)

    #Exibe um resumo dos filtros selecionados
    texto_selecao = 'ESCOLHA UMA PARTIDA PARA VISUALIZAR'
    
    if st.session_state.filtro_camp != 'Selecione':
    
        st.write('### SELEÇÃO REALIZADA')
        texto_selecao = st.session_state.filtro_camp
        
        if st.session_state.filtro_ano != 'Selecione':
            texto_selecao = st.session_state.filtro_camp + " > " + st.session_state.filtro_ano
            if st.session_state.filtro_partida != 'Selecione':
                texto_selecao = st.session_state.filtro_camp + " > " + st.session_state.filtro_ano + " > " + st.session_state.filtro_partida

    st.write(f'#### {texto_selecao}')

In [ ]:
#EXERCICIO 8
#Implemente um agente LangChain para realizar:
#   Consulta de eventos específicos da partida.
#   Geração de comparações entre dois jogadores.
#   Habilite o agente a responder perguntas como:
#       "Quem deu mais passes na partida?"
#       "Qual jogador teve mais finalizações no primeiro tempo?"

######### ACHO QUE AQUI EU TENHO CRIAR UMA TOOL PERSONALIZADA QUE FAÇA O CALCULO E RETORNE PARA O MODELO  (ASSISTIR A AULA EM QUE O FERNANDO ENSINA A CRIAR UMA TOOL PERSONALIZADA) 
######### LINK ENSINANDO A CRIAR A FERRAMENTA PERSONALIZADA https://python.langchain.com/docs/concepts/tools/

In [ ]:
# EXERCICIO 9
# Melhore a interface Streamlit, adicionando:
#   Filtros para explorar eventos específicos (e.g., apenas gols ou cartões).
#   Visualizações gráficas para estatísticas dos jogadores (e.g., gráficos de barras, tabelas interativas).
# Comparação direta entre dois jogadores selecionados.

############ PEGAR DO TP3, JÁ TEM ISSO PRONTO

In [41]:
df = sb.competitions()
#converter par dicionário

c:\Users\RodrigoPintoMesquita\Documents\GitHub\DR3_AT\.venv\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [47]:
df

,competition_name,season_name,competition_id,season_id
0,1. Bundesliga,2023/2024,9,281
1,1. Bundesliga,2015/2016,9,27
2,African Cup of Nations,2023,1267,107
3,Champions League,2018/2019,16,4
4,Champions League,2017/2018,16,1
...,...,...,...,...
69,UEFA Euro,2020,55,43
70,UEFA Europa League,1988/1989,35,75
71,UEFA Women's Euro,2022,53,106
72,Women's World Cup,2023,72,107


In [56]:
import requests
response = requests.get("http://localhost:8000/competitions")
response

<Response [200]>

In [64]:
response.json()

[{'competition_name': '1. Bundesliga',
  'season_name': '2023/2024',
  'competition_id': 9,
  'season_id': 281},
 {'competition_name': '1. Bundesliga',
  'season_name': '2015/2016',
  'competition_id': 9,
  'season_id': 27},
 {'competition_name': 'African Cup of Nations',
  'season_name': '2023',
  'competition_id': 1267,
  'season_id': 107},
 {'competition_name': 'Champions League',
  'season_name': '2018/2019',
  'competition_id': 16,
  'season_id': 4},
 {'competition_name': 'Champions League',
  'season_name': '2017/2018',
  'competition_id': 16,
  'season_id': 1},
 {'competition_name': 'Champions League',
  'season_name': '2016/2017',
  'competition_id': 16,
  'season_id': 2},
 {'competition_name': 'Champions League',
  'season_name': '2015/2016',
  'competition_id': 16,
  'season_id': 27},
 {'competition_name': 'Champions League',
  'season_name': '2014/2015',
  'competition_id': 16,
  'season_id': 26},
 {'competition_name': 'Champions League',
  'season_name': '2013/2014',
  'com

,competition_name,season_name,competition_id,season_id
0,1. Bundesliga,2023/2024,9,281
1,1. Bundesliga,2015/2016,9,27
2,African Cup of Nations,2023,1267,107
3,Champions League,2018/2019,16,4
4,Champions League,2017/2018,16,1
...,...,...,...,...
69,UEFA Euro,2020,55,43
70,UEFA Europa League,1988/1989,35,75
71,UEFA Women's Euro,2022,53,106
72,Women's World Cup,2023,72,107


In [70]:
response = requests.get(f"http://localhost:8000/matchanalysis/22912/Formal")

response.text

'"Análise da partida:\\n\\n**Primeiro Tempo:**\\n\\nO primeiro tempo foi marcado por pressão ofensiva do Liverpool.  Jordan Henderson e Mohamed Salah tiveram finalizações bloqueadas pela defesa do Tottenham.  No entanto, o Liverpool conseguiu abrir o placar com um gol de pênalti convertido por Mohamed Salah.  O placar ao final da primeira etapa foi Liverpool 1 x 0.\\n\\n**Segundo Tempo:**\\n\\nO segundo tempo apresentou um cenário diferente.  Ambas as equipes tiveram dificuldades em criar oportunidades claras de gol. Mohamed Salah (Liverpool), Christian Eriksen (Tottenham), Fábio Henrique Tavares (Liverpool), Lucas Moura e Dele Alli (ambos do Tottenham) tiveram chutes bloqueados pela defesa adversária.  Nenhum gol foi marcado na segunda etapa.\\n\\n\\n**Resumo Final:**\\n\\nA partida foi dominada pelo Liverpool no primeiro tempo, culminando em um gol de pênalti marcado por Mohamed Salah. O segundo tempo, porém, foi mais equilibrado, com poucas chances de gol para ambos os lados, result